### 테스트 영상데이터 분류
- 15개의 영상데이터를 키포인트와 프레임별 이미지로 나누어 데이터셋 생성하기
- 10프레임마다 만드는것으로 생성

In [5]:
import cv2
import mediapipe as mp
import json
import os
from collections import defaultdict

# mediapipe 라이브러리에서 필요한 모듈을 불러옴
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
mp_pose = mp.solutions.pose
mp_face_mesh = mp.solutions.face_mesh

In [6]:
# 랜드마크 데이터를 JSON 형식으로 변환하는 함수 정의
def serialize_landmarks(landmarks):
    serialized_landmarks = {}
    if landmarks:
        for id, landmark in enumerate(landmarks.landmark):
            serialized_landmarks[str(id)] = {
                'x': landmark.x,
                'y': landmark.y,
                'z': landmark.z,
                'visibility': landmark.visibility if hasattr(landmark, 'visibility') else None
            }
    return serialized_landmarks

In [12]:
for data_number in range(1,16) :    
    # 각 프레임에 대한 랜드마크 정보를 저장할 defaultdict 생성
    landmarks_data = defaultdict(dict)

    # 이미지를 저장할 디렉토리 지정
    images_dir = f'data/image/{data_number:02}'
    if not os.path.exists(images_dir):
        os.makedirs(images_dir)

    # 비디오 파일을 불러오기 위한 VideoCapture 객체 생성
    cap = cv2.VideoCapture(f'data/{data_number}.mp4')

    # 30프레임마다 이미지를 저장하기 위한 변수
    frame_interval = 10  # 저장할 프레임 간격 설정
    next_frame_to_save = 0  # 다음에 저장할 프레임 번호
    # Hands, Pose, FaceMesh 모델을 초기화하고 사용할 컨텍스트 매니저를 설정
    with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands, \
         mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose, \
         mp_face_mesh.FaceMesh(max_num_faces=1, min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:

        frame_num = 0
        # 비디오의 각 프레임에 대해 루프 실행
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                break  # 프레임을 불러오지 못하면 루프 종료

            # 이미지를 RGB로 변환 (mediapipe는 RGB 이미지를 사용)
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Hands, Pose, FaceMesh 처리
            hands_results = hands.process(image_rgb)
            pose_results = pose.process(image_rgb)
            face_results = face_mesh.process(image_rgb)

            # 랜드마크 직렬화 및 landmarks_data에 추가
            if frame_num == next_frame_to_save:
                if hands_results.multi_hand_landmarks:
                    for hand_landmarks in hands_results.multi_hand_landmarks:
                        landmarks_data[frame_num]['hands'] = serialize_landmarks(hand_landmarks)
                        # 손 랜드마크 그리기
                        # mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                if pose_results.pose_landmarks:
                    landmarks_data[frame_num]['pose'] = serialize_landmarks(pose_results.pose_landmarks)
                    # 포즈 랜드마크 그리기
                    #mp_drawing.draw_landmarks(image, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
                if face_results.multi_face_landmarks:
                    for face_landmarks in face_results.multi_face_landmarks:
                        landmarks_data[frame_num]['face'] = serialize_landmarks(face_landmarks)
                        # 얼굴 메시 랜드마크 그리기
                        #mp_drawing.draw_landmarks(image, face_landmarks, mp_face_mesh.FACEMESH_TESSELATION)

                # 처리된 프레임을 이미지 파일로 저장
                cv2.imwrite(os.path.join(images_dir, f'frame_{frame_num}.jpg'), image)
                next_frame_to_save += frame_interval  # 다음 저장할 프레임 갱신

            # 화면에 프레임 표시
            cv2.imshow('Frame', image)

            # 다음 프레임으로 넘어감
            frame_num += 1

            # ESC 키를 누르면 루프를 빠져나옴
            if cv2.waitKey(1) & 0xFF == 27:
                break

    # 모든 프레임에 대한 처리가 끝나면 결과를 JSON 파일로 저장
    json_path = os.path.join(f'data/json/{data_number:02}', f'{data_number:02}.json')  # JSON 파일을 저장할 경로
    if not os.path.exists(os.path.dirname(json_path)):
        os.makedirs(os.path.dirname(json_path))
    with open(json_path, 'w') as f:
        json.dump(landmarks_data, f, indent=4)

    # 비디오 캡처 객체 해제
    cap.release()
    # 모든 OpenCV 창 닫기
    cv2.destroyAllWindows()